In [1]:
import itertools
import typing

from icecream import ic
import opennre
import spacy

OPENNRE_MODEL: str = "wiki80_cnn_softmax"

SPACY_MODEL: str = "en_core_web_md"

nlp: spacy.Language = spacy.load(SPACY_MODEL)
nre: opennre.model.softmax_nn.SoftmaxNN = opennre.get_model(OPENNRE_MODEL)

2024-08-12 16:31:45,854 - root - INFO - Initializing word embedding with word2vec.


In [2]:
text: str = """
Eating processed red meat could be a significant risk factor for dementia, according to a large study that tracked more than 100,000 people over four decades.
Processed red meat has previously been shown to increase the risk of cancer, heart disease and type 2 diabetes. Now US researchers say they have uncovered a potential link to dementia.
The study also found that replacing processed red meat with healthier foods such as nuts, beans or tofu could help reduce the risk of dementia. The findings were presented at the Alzheimer’s Association international conference in the US.
The number of people living with dementia globally is forecast to nearly triple to 153 million by 2050, and studies looking at diet and risk of cognitive decline has become a focus of researchers.

In the latest research, experts studied the health of 130,000 nurses and other health workers working in the US. They were tracked for 43 years and provided data on their diet every 2 to 5 years.
The participants were asked how often they ate processed red meat including bacon, hotdogs, sausages, salami and other sandwich meat.
They were also asked about their consumption of nuts and legumes including peanut butter, peanuts, walnuts and other nuts, string beans, beans, peas, soy milk and tofu.

More than 11,000 cases of dementia were identified during the follow-up period.
Consuming two servings of processed red meat each week appeared to raise the risk of cognitive decline by 14% compared with those eating about three servings a month, the researchers reported.
The study also suggested that replacing one daily serving of processed red meat for a daily serving of nuts, beans or tofu every day could lower the risk of dementia by 23%.
    """.strip()

In [3]:
doc: spacy.tokens.doc.Doc = nlp(text)

In [8]:
OPENNRE_MIN_PROB: float = 0.5

STOP_RELS: typing.Set[ str ] = set([
    "has part",
])

for sent in doc.sents:
    ic(sent)
    
    for head, tail in itertools.combinations(list(sent.noun_chunks), 2): 
        rel, prob = nre.infer({
            "text": text,
            "h": { "pos": ( head.start_char, head.end_char) },
            "t": { "pos": ( tail.start_char, tail.end_char) },
        })

        if prob >= OPENNRE_MIN_PROB and rel not in STOP_RELS:
            ic(head, tail, rel, prob)

ic| sent: Eating processed red meat could be a significant risk factor for dementia, according to a large study that tracked more than 100,000 people over four decades.
ic| head: processed red meat
    tail: dementia
    rel: 'said to be the same as'
    prob: 0.7603546977043152
ic| head: a significant risk factor
    tail: that
    rel: 'main subject'
    prob: 0.6608046293258667
ic| head: a significant risk factor
    tail: more than 100,000 people
    rel: 'main subject'
    prob: 0.5330005884170532
ic| head: a large study
    tail: that
    rel: 'main subject'
    prob: 0.5174146294593811
ic| sent: Processed red meat has previously been shown to increase the risk of cancer, heart disease and type 2 diabetes.
ic| sent: Now US researchers say they have uncovered a potential link to dementia.
ic| sent: The study also found that replacing processed red meat with healthier foods such as nuts, beans or tofu could help reduce the risk of dementia.
ic| sent: The findings were presented at 